### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,35):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  680 non-null    object
dtypes: object(1)
memory usage: 5.4+ KB
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n10\n\n\nStory\n10\n\n\nAnimat...
1,\n\n\n\nOverall\n10\n\n\nStory\n10\n\n\nAnimat...
2,\n\n\n\nOverall\n10\n\n\nStory\n9\n\n\nAnimati...
3,\n\n\n\nOverall\n10\n\n\nStory\n9\n\n\nAnimati...
4,\n\n\n\nOverall\n6\n\n\nStory\n6\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


 Overall 10 Story 10 Animation 9 Sound 8 Character 10 Enjoyment 10 The lack of popularity surrounding Hunter x Hunter (2011) has always confused me, especially after catching up to it six months ago. For an anime that seems to do everything right, it isn't talked about a lot at a popular level. Neither has it been successful in garnering a fan base similar to that of SNK or SAO. I decided to write this review to do the show justice. If a show this good is still flying under your radar or if you are discouraged from watching this because of its "childish appearance" then reading this review is a must. Story: HxH is about a young boy named Gon who embarks on a journey to find his father. After learning that his father left him at a young age to become a Hunter, Gon decides to follow in his footsteps not only to find him, but also to see what was so special about the profession that made his father choose it over him. Although a story of finding one's father is simple, it is the path that

In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'the', 'lack', 'of', 'popularity', 'surrounding', 'hunter', 'hunter', 'has', 'always', 'confused', 'me', 'especially', 'after', 'catching', 'up', 'to', 'it', 'six', 'months', 'ago', 'for', 'an', 'anime', 'that', 'seems', 'to', 'do', 'everything', 'right', 'it', 'isn', 'talked', 'about', 'lot', 'at', 'popular', 'level', 'neither', 'has', 'it', 'been', 'successful', 'in', 'garnering', 'fan', 'base', 'similar', 'to', 'that', 'of', 'snk', 'or', 'sao', 'decided', 'to', 'write', 'this', 'review', 'to', 'do', 'the', 'show', 'justice', 'if', 'show', 'this', 'good', 'is', 'still', 'flying', 'under', 'your', 'radar', 'or', 'if', 'you', 'are', 'discouraged', 'from', 'watching', 'this', 'because', 'of', 'its', 'childish', 'appearance', 'then', 'reading', 'this', 'review', 'is', 'must', 'story', 'hxh', 'is', 'about', 'young', 'boy', 'named', 'gon', 'who', 'embarks', 'on', 'journey', 'to', 'find', 'his', 'father', 'after', 'learnin

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'after', 'reading', 'the', 'many', 'overwhelmingly', 'positive', 'reviews', 'about', 'this', 'show', 've', 'decided', 'to', 'write', 'more', 'realistic', 'honest', 'and', 'unbiased', 'opinion', 'driven', 'review', 'about', 'this', 'show', 'many', 'people', 'have', 'said', 'how', 'amazing', 'this', 'show', 'is', 'and', 'that', 'it', 'incredibly', 'unique', 'and', 'special', 'did', 'well', 'yes', 'and', 'no', 'am', 'probably', 'one', 'of', 'the', 'very', 'few', 'people', 'that', 'didn', 'consider', 'this', 'series', 'masterpiece', 'story', 'hunters', 'are', 'people', 'who', 'hunt', 'whether', 'it', 'is', 'criminals', 'treasures', 'rare', 'creatures', 'or', 'even', 'food', 'they', 'will', 'hunt', 'it', 'we', 'have', 'the', 'story', 'of', 'gon', 'young_boy', 'living', 'on', 'whale_island', 'aspiring', 'to', 'become', 'hunter', 'like', 'his', 'father', 'ging', 'legendary', 'hunter', 'who', 'abondoned', 'him', 'while', 'he'

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'lack', 'popularity', 'surround', 'hunter', 'hunter', 'always', 'confuse', 'especially', 'catch', 'month', 'ago', 'anime', 'seem', 'right', 'talk', 'lot', 'popular', 'level', 'successful', 'garnering', 'fan_base', 'similar', 'snk', 'sao', 'decide', 'write', 'review', 'show', 'justice', 'show', 'good', 'still', 'fly', 'radar', 'discourage', 'watch', 'childish', 'appearance', 'reading', 'review', 'story', 'name', 'gon', 'embark', 'journey', 'find', 'father', 'learn', 'father', 'leave', 'become', 'hunter', 'gon', 'decide', 'follow', 'footstep', 'find', 'also', 'see', 'special', 'profession', 'make', 'father', 'choose', 'story', 'find', 'father', 'simple', 'path', 'take', 'end', 'make', 'series', 'truly', 'special', 'hxh', 'make', 'several', 'arc', 'extremely', 'well', 'write', 'bring', 'good', 'part', 'series', 'write', 'hunter', 'hunter', 'boast', 'fine', 'write', 'world', 'shonen', 'depth', 'flow', 'story', 'enthral', 

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 4), (5, 2), (6, 2), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 13), (15, 2), (16, 2), (17, 4), (18, 4), (19, 7), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 2), (26, 1), (27, 2), (28, 3), (29, 26), (30, 1), (31, 1), (32, 4), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 3), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 7), (47, 1), (48, 1), (49, 4), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 2), (63, 1), (64, 1), (65, 2), (66, 1), (67, 3), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 3), (78, 16), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 3), (97, 1), (98, 1), (99, 1), (100, 3), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 

In [21]:
# Using LDA to develop a topic model with 3 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 3 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.004*"everytime" + 0.001*"dps" + 0.001*"expensive" + 0.001*"overdone" + 0.001*"dish" + 0.000*"distribution" + 0.000*"denial" + 0.000*"horrifically" + 0.000*"loli" + 0.000*"shelby"'), (1, '0.000*"hunter" + 0.000*"show" + 0.000*"activation" + 0.000*"boom" + 0.000*"abnormal" + 0.000*"divergent" + 0.000*"empathetic" + 0.000*"theorize" + 0.000*"fanta" + 0.000*"interruption"'), (2, '0.031*"que" + 0.012*"nao" + 0.009*"el" + 0.009*"por" + 0.008*"com" + 0.007*"anime" + 0.007*"ele" + 0.007*"hunter" + 0.007*"historia" + 0.007*"voce"'), (3, '0.000*"interruption" + 0.000*"activation" + 0.000*"stu" + 0.000*"undo" + 0.000*"abnormal" + 0.000*"divergent" + 0.000*"empathetic" + 0.000*"theorize" + 0.000*"fanta" + 0.000*"gary"'), (4, '0.029*"character" + 0.022*"arc" + 0.020*"hunter" + 0.019*"anime" + 0.016*"story" + 0.015*"show" + 0.011*"good" + 0.010*"gon" + 0.010*"watch" + 0.009*"well"'), (5, '0.000*"interruption" + 0.000*"activation" + 0.000*"stu" + 0.000*"undo" + 0.000*"abnormal" + 0.000*"diver

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.325255  0.145413       1        1  91.515008
7      0.178211 -0.163101       2        1   7.186171
2     -0.179620  0.148758       3        1   1.265930
0     -0.065628 -0.026904       4        1   0.018352
6     -0.064554 -0.026041       5        1   0.003742
1     -0.064554 -0.026041       6        1   0.003630
3     -0.064554 -0.026041       7        1   0.003618
5     -0.064554 -0.026041       8        1   0.003548, topic_info=            Term         Freq        Total Category  logprob  loglift
19         anime  3246.000000  3246.000000  Default  30.0000  30.0000
254       hunter  3531.000000  3531.000000  Default  29.0000  29.0000
453       series  1428.000000  1428.000000  Default  28.0000  28.0000
676      shounen   764.000000   764.000000  Default  27.0000  27.0000
230          gon  1863.000000  1863.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
25    appearance     0.000635    37.482149   Topic8  -9.2715  -0.7388
26   application     0.000635    12.782938   Topic8  -9.2715   0.3369
27         apply     0.000635    21.034737   Topic8  -9.2715  -0.1611
28      approach     0.000635    27.288382   Topic8  -9.2715  -0.4214
29           arc     0.000635  3779.983254   Topic8  -9.2715  -5.3524

[470 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.703414  ability
0         2  0.295828  ability
1         1  0.858705     able
1         2  0.141873     able
2         1  1.016977   absorb
...     ...       ...      ...
573       1  0.827240     work
573       2  0.173143     work
574       1  0.822022    world
574       2  0.172744    world
574       3  0.003971    world

[271 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 3, 1, 7, 2, 4, 6])

In [23]:
# Save the data frame as a CSV file
# job_datas.to_csv(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Programming Assignment 2\Chan_Matthew_INST447_PA2.csv')

In [24]:
# Week 9 Topic Modelling
# ner_corpus = ' '.join(ner_df['Name'])
# corp = open(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Week 9\corpus.txt',"w")
# corp.writelines(ner_corpus)
# corp.close()